In [1]:
import re
import pandas as pd

/gpfs/gibbs/project/gerstein/yj329/conda_envs/py3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('/gpfs/gibbs/pi/gerstein/yj329/epiPgene/parent/pgene_parent_mapping_Human103.txt', sep = '\t')
df['PgeneEnsembl103ID'] = df['PgeneEnsembl103ID'].apply(lambda x: x.split(".")[0])

In [3]:
df.head()

,PseudoPipeID,Chr,Start,End,Strand,PgeneEnsembl103ID,ParentProtein,ParentGene
0,PGOHUM00000289724,chr10,80394,86265,-,ENST00000423948,ENSP00000244174,ENSG00000124334
1,PGOHUM00000289726,chr10,4871901,4891975,-,ENST00000432950,ENSP00000370240,ENSG00000187134
2,PGOHUM00000289732,chr10,6071514,6071918,-,ENST00000397237,ENSP00000380156,ENSG00000144713
3,PGOHUM00000289733,chr10,6293100,6294684,-,ENST00000440192,ENSP00000260130,ENSG00000137575
4,PGOHUM00000289735,chr10,7295106,7295290,-,ENST00000421389,ENSP00000297564,ENSG00000164919


In [4]:
target_transcripts = set(df['PgeneEnsembl103ID'])
target_genes = set(df['ParentGene'])

In [ ]:
coords_db = {}

In [6]:
def parse_gtf_attribute(attr_str, key):
    start = attr_str.find(key + ' "')
    if start == -1: return None
    start += len(key) + 2
    end = attr_str.find('"', start)
    return attr_str[start:end]

In [7]:
GTF_FILE = "/gpfs/gibbs/pi/gerstein/yj329/epiPgene/gencode_v29/gencode.v29.annotation.gtf"
with open(GTF_FILE, 'r') as f:
    for line in f:
        if line.startswith('#'): continue
        parts = line.strip().split('\t')
        if len(parts) < 9: continue
        feature_type = parts[2] # gene or transcript
        attributes = parts[8]

        if feature_type == 'transcript':
            tid = parse_gtf_attribute(attributes, 'transcript_id')
            geneType =  parse_gtf_attribute(attributes, 'gene_type')
            
            match = re.search(r"(processed|unprocessed)", geneType)
            if match:
                geneType = match.group(1)
                tid_clean = tid.split(".")[0] 
                if tid_clean in target_transcripts:
                    coords_db[tid_clean] = {
                        'chr': parts[0],
                        'start': int(parts[3]),
                        'end': int(parts[4]),
                        'strand': parts[6],
                        'geneType': geneType.capitalize()
                    }
        
        elif feature_type == 'gene':
            gid = parse_gtf_attribute(attributes, 'gene_id')
            if gid:
                gid_clean = gid.split('.')[0]
                if gid_clean in target_genes:
                    coords_db[gid_clean] = {
                        'chr': parts[0],
                        'start': int(parts[3]),
                        'end': int(parts[4]),
                        'strand': parts[6]}
                    

In [8]:
def get_upstream_1kb(fasta_obj, coord_info):

    chrom = coord_info['chr']
    strand = coord_info['strand']
    
    gtf_start = coord_info['start']
    gtf_end = coord_info['end']
    
    seq_str = ""
    
    try:
        if strand == '+':
            fetch_start = gtf_start - 1 - 1000
            fetch_end = gtf_start - 1
            if fetch_start < 0: fetch_start = 0
            
            seq_str = fasta_obj.fetch(chrom, fetch_start, fetch_end)
            
        elif strand == '-':
            fetch_start = gtf_end
            fetch_end = gtf_end + 1000
            
            seq_str = fasta_obj.fetch(chrom, fetch_start, fetch_end)
            seq_str = str(Seq(seq_str).reverse_complement())
            
    except KeyError:
        print(f"Error: Chromosome {chrom} not found in FASTA.")
        return None
    except Exception as e:
        print(f"Error fetching sequence: {e}")
        return None

    return seq_str.upper()

In [9]:
from Bio.Seq import Seq
import pysam
fasta = pysam.FastaFile('/gpfs/gibbs/pi/gerstein/yj329/epiPgene/gencode_v29/hg38.fa')

In [28]:
from Bio import Align

def pairwise_alignment(seq_pgene, seq_parent, MIN_FRAG_LEN = 10):
    aligner = Align.PairwiseAligner()
    aligner.mode = 'local'
    
    aligner.match_score = 2       # Match +2
    aligner.mismatch_score = -3   # Mismatch -3 
    aligner.open_gap_score = -5   # Open Gap -5
    aligner.extend_gap_score = -2 # Extend Gap -2
        
    alignments = aligner.align(seq_pgene, seq_parent)
    try:
        best_aln = alignments[0]
    except IndexError:
        return {"Inclusion_Pct": 0, "Identity_Pct": 0, "Coordinates": "[]"}

    parent_aligned_regions = best_aln.aligned[1]
    ## filtering on alignment
    valid_parent_len = 0     
    valid_regions_list = []  
    for start, end in parent_aligned_regions:
        fragment_len = end - start
        if fragment_len >= MIN_FRAG_LEN:
            valid_parent_len += fragment_len
            valid_regions_list.append((start, end)) 
    
    # coverage       
    parent_total_len = len(seq_parent)    
    inclusion_pct = (valid_parent_len / parent_total_len) * 100 if parent_total_len > 0 else 0
    # identity
    if valid_parent_len == 0:
        identity_pct = 0
    else:
        counts = best_aln.counts()
        matches = counts.identities 
        aln_len = best_aln.length  
        #print(f"Valid Len: {valid_parent_len}, Total Aln Len: {aln_len}, Matches: {matches}")
        identity_pct = (matches / aln_len) * 100 if aln_len > 0 else 0

    return {
        "Inclusion_Pct": round(inclusion_pct, 2),
        "Identity_Pct": round(identity_pct, 2),
        "Coordinates": str(valid_regions_list)}

In [31]:
from tqdm import tqdm

results = []
for idx, row in tqdm(df.iterrows()):
    pgene_id = row['PgeneEnsembl103ID'].split('.')[0]
    parent_id = row['ParentGene']
    
    pgene_coords = coords_db.get(pgene_id)
    parent_coords = coords_db.get(parent_id)

    metrics = {"Inclusion_Pct": 0, "Identity_Pct": 0, "Coordinates": "[]"}
    
    if pgene_coords and parent_coords:
        seq_pgene = get_upstream_1kb(fasta, pgene_coords)
        seq_parent = get_upstream_1kb(fasta, parent_coords)   
        metrics = pairwise_alignment(seq_pgene, seq_parent)
        
        res_row = row.to_dict()
        res_row.update(metrics)
        res_row.update({'geneType': pgene_coords.get('geneType')})
        results.append(res_row)    

13241it [04:09, 53.16it/s]


In [32]:
final_df = pd.DataFrame(results)
final_df = final_df.dropna().reset_index(drop=True)
final_df = final_df[final_df['Inclusion_Pct']!=0]

In [80]:
final_df.to_csv('/gpfs/gibbs/pi/gerstein/yj329/epiPgene/parent/tss1kb_comparison/tss1kb_inclusion_identity.txt', index=None, sep = '\t')